In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
data = pd.read_csv('data/class.csv')
code_list=[]
for i in [i for i in data.columns[1:]]:
    code_list.append(i)

-----------------------

In [11]:
## data var ##
code = 'a5930'
n_test_set = 10 # 20개는 테스트 셋 사용할 갯수
est_data = 2 # n 개의 단어를 잘라줌(n개를 예측하는 모형)
batch_size = 5 #한번에 5개의 데이터를 사용

---------------------

In [12]:
train = np.array(data[code][:-n_test_set]) 
test = data[-n_test_set:-est_data][code]
class_set = sorted(set(train)) 
class_set.append('NO')
word_idx = {u:i for i, u in enumerate(class_set)}
idx_word = np.array(class_set)
text_as_int = np.array([word_idx[c] for c in train])
seq_length = n_test_set 
examples_per_epoch = len(text_as_int) // seq_length
sentence_dataset = tf.data.Dataset.from_tensor_slices(text_as_int) 
sentence_dataset = sentence_dataset.batch(seq_length+est_data, drop_remainder=True) 
def split(data): 
    return [data[:-est_data], data[-est_data]]
train_dataset = sentence_dataset.map(split) 
BATCH_SIZE = batch_size 
steps_per_epoch = examples_per_epoch // BATCH_SIZE
BUFFER_SIZE = 10000
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
total_words = len(class_set)
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_words,n_test_set, input_length=seq_length),
    tf.keras.layers.LSTM(units=64, return_sequences=True),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.LSTM(units=32),
    tf.keras.layers.Dense(total_words, activation='softmax')])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
from tensorflow.keras.preprocessing.sequence import pad_sequences
def testmodel(epoch, logs):
    if epoch % 5 != 0 and epoch != 49:
        return
    test_sentence = data[code][-n_test_set:]
    next_words = est_data
    for _ in range(next_words):
        test_text_X = test_sentence[-seq_length:]
        test_text_X = np.array([word_idx[c] if c in word_idx else word_idx['NO'] for c in test_text_X])
        test_text_X = pad_sequences([test_text_X], maxlen=seq_length, padding='pre', value=word_idx['NO'])
        output_idx = model.predict_classes(test_text_X)
        test_sentence += ' ' + idx_word[output_idx[0]]
testmodelcb = tf.keras.callbacks.LambdaCallback(on_epoch_end=testmodel)
history = model.fit(train_dataset.repeat(), epochs=100, steps_per_epoch=steps_per_epoch, callbacks=[testmodelcb], verbose=2)
from tensorflow.keras.preprocessing.sequence import pad_sequences
next_class = est_data
for _ in range(next_class):
    test_text_X = test_sentence.split(' ')[-seq_length:]
    test_text_X = np.array([word_idx[c] if c in word_idx else word_idx['NO'] for c in test_text_X])
    test_text_X = pad_sequences([test_text_X], maxlen=seq_length, padding='pre', value=word_idx['NO'])
    output_idx = model.predict_classes(test_text_X)
    test_sentence += ' ' + idx_word[output_idx[0]]

Train for 4 steps
Epoch 1/100
4/4 - 3s - loss: 2.7742 - accuracy: 0.0000e+00
Epoch 2/100
4/4 - 0s - loss: 2.7657 - accuracy: 0.1000
Epoch 3/100
4/4 - 0s - loss: 2.7583 - accuracy: 0.1000
Epoch 4/100
4/4 - 0s - loss: 2.7497 - accuracy: 0.1000
Epoch 5/100
4/4 - 0s - loss: 2.7394 - accuracy: 0.1000
Epoch 6/100
4/4 - 0s - loss: 2.7218 - accuracy: 0.1000
Epoch 7/100
4/4 - 0s - loss: 2.6992 - accuracy: 0.1000
Epoch 8/100
4/4 - 0s - loss: 2.6703 - accuracy: 0.1000
Epoch 9/100
4/4 - 0s - loss: 2.6130 - accuracy: 0.1000
Epoch 10/100
4/4 - 0s - loss: 2.5393 - accuracy: 0.1000
Epoch 11/100
4/4 - 0s - loss: 2.4942 - accuracy: 0.1000
Epoch 12/100
4/4 - 0s - loss: 2.4495 - accuracy: 0.1000
Epoch 13/100
4/4 - 0s - loss: 2.4328 - accuracy: 0.1000
Epoch 14/100
4/4 - 0s - loss: 2.4039 - accuracy: 0.1500
Epoch 15/100
4/4 - 0s - loss: 2.3729 - accuracy: 0.1500
Epoch 16/100
4/4 - 0s - loss: 2.3794 - accuracy: 0.1500
Epoch 17/100
4/4 - 0s - loss: 2.3596 - accuracy: 0.1000
Epoch 18/100
4/4 - 0s - loss: 2.352

In [14]:
print('4,5번 값 = 예측값')
print('predict data >>>',test_sentence[-20:])
print('actual data  >>>','', str(data['a28260'][-5:].tolist()).replace("[","").replace("]","").replace(",","").replace("'",""))

4,5번 값 = 예측값
predict data >>>  c20 c20 c18 c18 c08
actual data  >>>  c25 c08 c20 c11 c01
